In [0]:
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn .naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support
import pickle

import json
import numpy as np
import pandas as pd
import os
from scipy.fftpack import fft, ifft

from dtw import dtw

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn .naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support
import pickle
import pandas as pd
import glob
from scipy import fftpack as fft
import pywt
import numpy as np

In [0]:
def dynamicTimeWarpDistance(df):
    manhattan_distance = lambda x, y: np.abs(x - y)
    dtw_distance = []
    for i in range(df.shape[0]-1):
      temp, _, _, _ = dtw(df.iloc[i,:52], df.iloc[i+1,:52], dist=manhattan_distance)
      dtw_distance.append(temp)
    dtw_distance.append(0)
    return pd.DataFrame(data=np.array(dtw_distance))

In [0]:
def fastFourierTransform(df):
    fastFourierTransform_df = fft.rfft(df, n=5, axis=1)
    return pd.DataFrame(data=fastFourierTransform_df)

In [0]:
def discreteWaveletTransform(df):
    cA, cD = pywt.dwt(df, 'haar')
    return pd.DataFrame(cA),pd.DataFrame(cD)

In [0]:
def universalNormalization(df):
    cols1=['nose_x', 'nose_y','leftEye_x', 'leftEye_y','rightEye_x', 'rightEye_y','rightShoulder_x', 'rightShoulder_y', 'leftWrist_x',
       'leftWrist_y', 'rightWrist_x', 'rightWrist_y']
    
    cols2=['leftWrist_x', 'leftWrist_y', 'rightWrist_x', 'rightWrist_y']
    
    X=df[cols1]
    
    X['leftWrist_x']=(X['leftWrist_x']-X['nose_x'])/(X['leftEye_x']-X['rightEye_x'])
    X['leftWrist_y']=(X['leftWrist_y']-X['nose_y'])/(X['nose_y']-X['rightShoulder_y'])
    
    X['rightWrist_x']=(X['rightWrist_x']-X['nose_x'])/(X['leftEye_x']-X['rightEye_x'])
    X['rightWrist_y']=(X['rightWrist_y']-X['nose_y'])/(X['nose_y']-X['rightShoulder_y'])
    
    data=X[cols2]
    
    fft_df = fastFourierTransform(data)
    ca,cd = discreteWaveletTransform(data)
    dtw_df = dynamicTimeWarpDistance(data)
    
    features_data=pd.concat([X,fft_df,ca,cd, dtw_df],axis=1,ignore_index=True)
    
    labelValue=df['label']
    
    return features_data, labelValue

In [0]:
df = pd.read_csv("key_points.csv")
df = df.drop(columns="Frames#")

X,Y = universalNormalization(df)

In [0]:
def kFoldCrossValidation(X,Y):

    kfold = model_selection.KFold(n_splits=10, random_state=42)
    # model = LogisticRegression()   
    # model = GaussianNB()  
    # model = svm.NuSVC(gamma='auto')   
    # model = KNeighborsClassifier(n_neighbors=10, algorithm='auto' )  
    # model = RandomForestClassifier()   
    # model = DecisionTreeClassifier()   
    
    results = model_selection.cross_val_score(model, X, Y, cv=kfold)
    print("Accuracy after KFold Cross Validation: %.3f%% " % (results.mean()*100.0)) 

In [0]:
kFoldCrossValidation(X,Y)

In [0]:
def finalModelTestTrain(X,Y):
    
    #select maximum accuracy model
    # model = LogisticRegression()   
    model = GaussianNB()  
    # model = svm.NuSVC(gamma='auto')   
    # model = KNeighborsClassifier(n_neighbors=10, algorithm='auto' )  
    # model = RandomForestClassifier()   
    # model = DecisionTreeClassifier()   

    test_size = 0.25
    seed = 0
    
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

    model.fit(X_train, Y_train)

    filename = 'gnb.pkl'
    pickle.dump(model, open(filename, 'wb'))
     

    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred=loaded_model.predict(X_test)
    
    result = loaded_model.score(X_test, Y_test)
    print("Accuracy of Trained Model: %.3f%% " % (result*100.0))
    print("Precison, Recall, F Socre:")
    print(precision_recall_fscore_support(Y_test, y_pred, average='macro'))

In [0]:
finalModelTestTrain(X,Y)